In [44]:
import numpy as np
data = np.load("/home/h/Opengate/Post_process/Outputs/coincidence_pairs.npz")

In [45]:
import numpy as np
from numba import njit, prange

@njit
def siddon_single_lor(p1, p2, n_voxels, voxel_size, half_fov):
    """
    Siddon's algorithm for a single LOR.
    Returns arrays of (voxel_indices, intersection_lengths).
    """
    # Convert to voxel coordinate system (0 to n_voxels)
    p1_v = np.empty(3)
    p2_v = np.empty(3)
    for i in range(3):
        p1_v[i] = (p1[i] + half_fov) / voxel_size
        p2_v[i] = (p2[i] + half_fov) / voxel_size
    
    # Direction vector
    d = np.empty(3)
    for i in range(3):
        d[i] = p2_v[i] - p1_v[i]
    
    # LOR length in voxel units
    lor_length = np.sqrt(d[0]**2 + d[1]**2 + d[2]**2)
    
    if lor_length < 1e-10:
        return np.empty((0, 3), dtype=np.int32), np.empty(0, dtype=np.float32)
    
    # Compute alpha_min and alpha_max (parametric intersection with volume)
    alpha_min = 0.0
    alpha_max = 1.0
    
    for i in range(3):
        if abs(d[i]) > 1e-10:
            a1 = (0 - p1_v[i]) / d[i]
            a2 = (n_voxels - p1_v[i]) / d[i]
            
            if a1 > a2:
                a1, a2 = a2, a1
            
            alpha_min = max(alpha_min, a1)
            alpha_max = min(alpha_max, a2)
    
    if alpha_min >= alpha_max:
        return np.empty((0, 3), dtype=np.int32), np.empty(0, dtype=np.float32)
    
    # Compute all alpha values at voxel boundaries for each axis
    # Maximum possible intersections
    max_intersections = 3 * n_voxels + 3
    alphas = np.empty(max_intersections, dtype=np.float64)
    n_alphas = 0
    
    # Add alpha_min and alpha_max
    alphas[n_alphas] = alpha_min
    n_alphas += 1
    alphas[n_alphas] = alpha_max
    n_alphas += 1
    
    # For each axis, add alpha values at voxel boundaries
    for axis in range(3):
        if abs(d[axis]) < 1e-10:
            continue
        
        # Determine range of planes crossed
        if d[axis] > 0:
            i_min = int(np.ceil(p1_v[axis] + alpha_min * d[axis]))
            i_max = int(np.floor(p1_v[axis] + alpha_max * d[axis]))
        else:
            i_min = int(np.ceil(p1_v[axis] + alpha_max * d[axis]))
            i_max = int(np.floor(p1_v[axis] + alpha_min * d[axis]))
        
        # Clamp to valid range
        i_min = max(0, min(n_voxels, i_min))
        i_max = max(0, min(n_voxels, i_max))
        
        # Add alpha for each plane crossing
        for i in range(i_min, i_max + 1):
            alpha = (i - p1_v[axis]) / d[axis]
            if alpha_min < alpha < alpha_max:
                alphas[n_alphas] = alpha
                n_alphas += 1
    
    # Sort alphas
    alphas_sorted = np.sort(alphas[:n_alphas])
    
    # Remove duplicates (within tolerance)
    unique_alphas = np.empty(n_alphas, dtype=np.float64)
    n_unique = 0
    prev = -1e10
    for i in range(len(alphas_sorted)):
        if alphas_sorted[i] - prev > 1e-10:
            unique_alphas[n_unique] = alphas_sorted[i]
            n_unique += 1
            prev = alphas_sorted[i]
    
    if n_unique < 2:
        return np.empty((0, 3), dtype=np.int32), np.empty(0, dtype=np.float32)
    
    # Compute voxel indices and intersection lengths
    n_voxels_hit = n_unique - 1
    voxel_indices = np.empty((n_voxels_hit, 3), dtype=np.int32)
    lengths = np.empty(n_voxels_hit, dtype=np.float32)
    
    count = 0
    for i in range(n_unique - 1):
        alpha_mid = (unique_alphas[i] + unique_alphas[i + 1]) / 2.0
        
        # Voxel at midpoint
        ix = int(p1_v[0] + alpha_mid * d[0])
        iy = int(p1_v[1] + alpha_mid * d[1])
        iz = int(p1_v[2] + alpha_mid * d[2])
        
        # Bounds check
        if 0 <= ix < n_voxels and 0 <= iy < n_voxels and 0 <= iz < n_voxels:
            voxel_indices[count, 0] = ix
            voxel_indices[count, 1] = iy
            voxel_indices[count, 2] = iz
            
            # Intersection length in mm
            lengths[count] = (unique_alphas[i + 1] - unique_alphas[i]) * lor_length * voxel_size
            count += 1
    
    return voxel_indices[:count], lengths[:count]


@njit(parallel=True)
def siddon_backproject(xyz1, xyz2, voxel_size, fov, weighted=True):
    """
    Full Siddon backprojection for all LORs.
    
    weighted=True:  increment by intersection length (for MLEM)
    weighted=False: increment by 1 per voxel hit
    """
    n_voxels = int(fov / voxel_size)
    half_fov = fov / 2.0
    n_lors = xyz1.shape[0]
    
    # Use separate images per thread to avoid race conditions, then sum
    # For simplicity here, we'll use a single image with atomic-like updates
    # (Numba prange handles this reasonably for float addition)
    image = np.zeros((n_voxels, n_voxels, n_voxels), dtype=np.float32)
    
    for i in prange(n_lors):
        p1 = xyz1[i]
        p2 = xyz2[i]
        
        voxels, lengths = siddon_single_lor(p1, p2, n_voxels, voxel_size, half_fov)
        
        for j in range(len(lengths)):
            ix = voxels[j, 0]
            iy = voxels[j, 1]
            iz = voxels[j, 2]
            
            if weighted:
                image[ix, iy, iz] += lengths[j]
            else:
                image[ix, iy, iz] += 1.0
    
    return image


def siddon_backproject_chunked(xyz1, xyz2, voxel_size=2.0, fov=200.0, 
                                weighted=True, chunk_size=500_000, verbose=True):
    """
    Chunked wrapper for memory efficiency and progress reporting.
    """
    n_voxels = int(fov / voxel_size)
    n_lors = len(xyz1)
    
    if verbose:
        print(f"Siddon backprojection:")
        print(f"  LORs: {n_lors:,}")
        print(f"  Image: {n_voxels}³ voxels ({voxel_size}mm)")
        print(f"  Weighted: {weighted}")
    
    image = np.zeros((n_voxels, n_voxels, n_voxels), dtype=np.float32)
    
    for start in range(0, n_lors, chunk_size):
        end = min(start + chunk_size, n_lors)
        
        chunk_image = siddon_backproject(
            xyz1[start:end].astype(np.float64),
            xyz2[start:end].astype(np.float64),
            voxel_size, fov, weighted
        )
        
        image += chunk_image
        
        if verbose and (start // chunk_size) % 5 == 0:
            print(f"  {end:,}/{n_lors:,} ({100*end/n_lors:.1f}%)")
    
    if verbose:
        print(f"Done! Non-zero voxels: {np.count_nonzero(image):,}")
    
    return image

# Run Siddon backprojection
# weighted=True  -> intersection length (for MLEM)
# weighted=False -> count per voxel (for visualization)
image = siddon_backproject_chunked(
    data['xyz1'], data['xyz2'],
    voxel_size=2.0,
    fov=200.0,
    weighted=True,
    chunk_size=100_000
)

# Save
# np.save("siddon_backprojection.npy", image)

Siddon backprojection:
  LORs: 1,144,222
  Image: 100³ voxels (2.0mm)
  Weighted: True
  100,000/1,144,222 (8.7%)
  600,000/1,144,222 (52.4%)
  1,100,000/1,144,222 (96.1%)
Done! Non-zero voxels: 998,886


In [46]:
from itkwidgets import view

view(image)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…